# PyTorch Optimizer Comparison on SOEN SingleDendrite Model

This notebook compares different PyTorch optimizers on a **minimal SOEN model** with `SingleDendrite` neurons.

## Model Architecture
- **Input Layer**: 1D flux input
- **Hidden Layer**: 1 SingleDendrite neuron (physical SOEN neuron)
- **Output Layer**: 1D readout

## Trainable Parameters: Exactly 2
- **J_input**: Input connection weight (1×1 = 1 parameter)
- **J_output**: Output connection weight (1×1 = 1 parameter)

## Task
Linear regression: learn to map input flux to target output through SingleDendrite dynamics.

## Optimizers Compared
1. **SGD** - Vanilla Stochastic Gradient Descent
2. **SGD + Momentum** - SGD with momentum
3. **Adam** - Adaptive Moment Estimation
4. **AdamW** - Adam with decoupled weight decay
5. **RMSprop** - Root Mean Square Propagation

---

In [ ]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

from soen_toolkit.core import (
    ConnectionConfig,
    LayerConfig,
    SimulationConfig,
    SOENModelCore,
)

# Set random seed for reproducibility
torch.manual_seed(42)
np.random.seed(42)

print(f"PyTorch version: {torch.__version__}")

## 1. Generate Synthetic Regression Data

Create a simple linear regression dataset where the target is a scaled version of the input.
The SOEN model will learn to approximate this mapping through its SingleDendrite dynamics.

In [ ]:
def generate_regression_data(n_samples=100, seq_len=30, noise_std=0.01):
    """
    Generate regression data: constant input signals with scalar targets.
    
    The SOEN model receives a constant flux input and should produce
    an output that matches the target (a linear function of the input).
    
    Returns:
        X: [n_samples, seq_len, 1] - input flux sequences (constant per sample)
        y: [n_samples, 1] - regression targets
    """
    # Ground truth relationship: y = 2.0 * x + 0.5
    TRUE_SCALE = 2.0
    TRUE_OFFSET = 0.5
    
    # Generate input values (flux magnitudes)
    x_values = torch.linspace(0.05, 0.2, n_samples)  # Within SOEN operating range
    
    # Create constant input sequences
    X = x_values.unsqueeze(1).unsqueeze(2).expand(-1, seq_len, 1).clone()
    
    # Add small noise to inputs
    X = X + noise_std * torch.randn_like(X)
    
    # Generate targets (what the final state should approximate)
    y = TRUE_SCALE * x_values + TRUE_OFFSET
    y = y.unsqueeze(1)  # [n_samples, 1]
    
    return X, y, TRUE_SCALE, TRUE_OFFSET

# Generate data
N_SAMPLES = 100
SEQ_LEN = 30
X_data, y_data, TRUE_SCALE, TRUE_OFFSET = generate_regression_data(N_SAMPLES, SEQ_LEN)

print(f"Input shape: {X_data.shape}")
print(f"Target shape: {y_data.shape}")
print(f"Ground truth: y = {TRUE_SCALE} * x + {TRUE_OFFSET}")
print(f"Input range: [{X_data.mean(dim=1).min():.3f}, {X_data.mean(dim=1).max():.3f}]")
print(f"Target range: [{y_data.min():.3f}, {y_data.max():.3f}]")

# Visualize data
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Input-output relationship
x_means = X_data.mean(dim=1).squeeze().numpy()
ax1 = axes[0]
ax1.scatter(x_means, y_data.squeeze().numpy(), alpha=0.6, label='Data')
ax1.plot(x_means, TRUE_SCALE * x_means + TRUE_OFFSET, 'r-', linewidth=2, 
         label=f'True: y = {TRUE_SCALE}x + {TRUE_OFFSET}')
ax1.set_xlabel('Input flux (mean)')
ax1.set_ylabel('Target')
ax1.set_title('Regression Task')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Sample input sequences
ax2 = axes[1]
for i in [0, 25, 50, 75, 99]:
    ax2.plot(X_data[i, :, 0].numpy(), label=f'Sample {i}')
ax2.set_xlabel('Time step')
ax2.set_ylabel('Input flux')
ax2.set_title('Sample Input Sequences')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 2. Build Minimal SOEN Model (2 Trainable Parameters)

Architecture: 1D → 1D (SingleDendrite) → 1D

- **J_input** (1×1): Weight from input to SingleDendrite neuron
- **J_output** (1×1): Weight from SingleDendrite to output

Total: **2 trainable parameters**

The SingleDendrite dynamics are:
$$\frac{ds}{dt} = \gamma^+ \cdot g(\phi) - \gamma^- \cdot s$$

where $\phi = J_{input} \cdot x$ is the weighted input flux.

In [ ]:
def build_minimal_soen_model(dt=50.0, init_j_in=0.15, init_j_out=1.0):
    """
    Build a minimal SOEN model with exactly 2 trainable parameters.
    
    Architecture: 1 (input) → 1 (SingleDendrite) → 1 (output)
    Trainable: J_input (1x1) and J_output (1x1) = 2 parameters
    """
    sim_cfg = SimulationConfig(
        dt=dt,
        input_type="state",
        track_phi=False,
        track_power=False,
    )
    
    # Layer 0: Single input channel
    layer0 = LayerConfig(
        layer_id=0,
        layer_type="Input",
        description="Input flux",
        params={"dim": 1},
    )
    
    # Layer 1: Single SingleDendrite neuron
    layer1 = LayerConfig(
        layer_id=1,
        layer_type="SingleDendrite",
        description="SOEN neuron",
        params={
            "dim": 1,  # Single neuron
            "solver": "FE",
            "source_func": "Heaviside_fit_state_dep",
            "phi_offset": 0.02,
            "bias_current": 1.98,
            "gamma_plus": 0.0005,  # Faster dynamics for shorter sequences
            "gamma_minus": 1e-6,
        },
    )
    
    # Layer 2: Single output
    layer2 = LayerConfig(
        layer_id=2,
        layer_type="Input",
        description="Output",
        params={"dim": 1},
    )
    
    layers = [layer0, layer1, layer2]
    
    # Connection 0→1: Input weight (1 parameter)
    conn01 = ConnectionConfig(
        from_layer=0,
        to_layer=1,
        connection_type="all_to_all",  # 1x1 = 1 parameter
        learnable=True,
        params={
            "init": "constant",
            "value": init_j_in,
            "constraints": {"min": -0.5, "max": 0.5},
        },
    )
    
    # Connection 1→2: Output weight (1 parameter)
    conn12 = ConnectionConfig(
        from_layer=1,
        to_layer=2,
        connection_type="all_to_all",  # 1x1 = 1 parameter
        learnable=True,
        params={
            "init": "constant",
            "value": init_j_out,
            "constraints": {"min": -5.0, "max": 5.0},
        },
    )
    
    connections = [conn01, conn12]
    
    model = SOENModelCore(
        sim_config=sim_cfg,
        layers_config=layers,
        connections_config=connections,
    )
    
    return model

# Build and verify model
test_model = build_minimal_soen_model()
print("Model Structure:")
print(f"  Layers: {[l.dim for l in test_model.layers]}")

# Count trainable parameters
trainable_params = [(name, p) for name, p in test_model.named_parameters() if p.requires_grad]
total_trainable = sum(p.numel() for _, p in trainable_params)

print(f"\nTrainable Parameters: {total_trainable}")
for name, param in trainable_params:
    print(f"  {name}: shape={list(param.shape)}, value={param.item():.4f}")

In [ ]:
# Visualize the model architecture
test_model.visualize(show_descriptions=True, theme="modern")

## 3. Visualize Loss Landscape

Compute MSE loss over a grid of (J_input, J_output) values to see the optimization landscape.

In [ ]:
def compute_soen_loss(model, X, y):
    """Compute MSE loss for SOEN model."""
    model.eval()
    with torch.no_grad():
        final_hist, _ = model(X)
        # Take final time step as output
        output = final_hist[:, -1, :]  # [batch, 1]
        loss = ((output - y) ** 2).mean()
    return loss.item()

def compute_loss_landscape_soen(X_data, y_data, j_in_range, j_out_range, resolution=30):
    """
    Compute loss landscape by evaluating SOEN model at grid of (J_in, J_out) values.
    """
    j_in_vals = np.linspace(j_in_range[0], j_in_range[1], resolution)
    j_out_vals = np.linspace(j_out_range[0], j_out_range[1], resolution)
    J_IN, J_OUT = np.meshgrid(j_in_vals, j_out_vals)
    
    loss_surface = np.zeros_like(J_IN)
    
    # Use subset of data for faster computation
    X_subset = X_data[::5]  # Every 5th sample
    y_subset = y_data[::5]
    
    for i in range(resolution):
        for j in range(resolution):
            j_in, j_out = J_IN[i, j], J_OUT[i, j]
            model = build_minimal_soen_model(init_j_in=j_in, init_j_out=j_out)
            loss_surface[i, j] = compute_soen_loss(model, X_subset, y_subset)
    
    return J_IN, J_OUT, loss_surface

# Compute loss landscape
print("Computing loss landscape (this may take a moment)...")
J_IN_RANGE = (0.05, 0.35)
J_OUT_RANGE = (0.5, 3.5)
J_IN, J_OUT, loss_surface = compute_loss_landscape_soen(
    X_data, y_data, J_IN_RANGE, J_OUT_RANGE, resolution=25
)
print("Done!")

# Find approximate minimum
min_idx = np.unravel_index(np.argmin(loss_surface), loss_surface.shape)
OPT_J_IN = J_IN[min_idx]
OPT_J_OUT = J_OUT[min_idx]
print(f"\nApproximate optimum: J_in={OPT_J_IN:.3f}, J_out={OPT_J_OUT:.3f}")
print(f"Minimum loss: {loss_surface[min_idx]:.6f}")

In [ ]:
# Plot loss landscape
fig = plt.figure(figsize=(14, 5))

# 3D surface
ax1 = fig.add_subplot(121, projection='3d')
ax1.plot_surface(J_IN, J_OUT, loss_surface, cmap='viridis', alpha=0.8)
ax1.scatter([OPT_J_IN], [OPT_J_OUT], [loss_surface[min_idx]], 
            color='red', s=100, marker='*', label='Optimum')
ax1.set_xlabel('J_input')
ax1.set_ylabel('J_output')
ax1.set_zlabel('MSE Loss')
ax1.set_title('SOEN Loss Landscape (3D)')

# Contour plot
ax2 = fig.add_subplot(122)
contour = ax2.contour(J_IN, J_OUT, loss_surface, levels=30, cmap='viridis')
ax2.clabel(contour, inline=True, fontsize=8)
ax2.scatter([OPT_J_IN], [OPT_J_OUT], color='red', s=100, marker='*', 
            zorder=5, label=f'Optimum ({OPT_J_IN:.2f}, {OPT_J_OUT:.2f})')
ax2.set_xlabel('J_input')
ax2.set_ylabel('J_output')
ax2.set_title('SOEN Loss Landscape (Contour)')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 4. Training Function with Trajectory Tracking

Train the SOEN model and record the parameter trajectory in (J_in, J_out) space.

In [ ]:
def get_trainable_params(model):
    """Extract J_input and J_output values from model."""
    params = {}
    for name, p in model.named_parameters():
        if p.requires_grad:
            if 'connection_0_to_1' in name:
                params['j_in'] = p.item()
            elif 'connection_1_to_2' in name:
                params['j_out'] = p.item()
    return params['j_in'], params['j_out']

def train_soen_model(optimizer_class, optimizer_kwargs, X_data, y_data,
                     n_epochs=100, init_j_in=0.1, init_j_out=0.8, verbose=False):
    """
    Train a minimal SOEN model and track the optimization trajectory.
    
    Returns:
        Dict with losses, trajectory, final parameters
    """
    # Build fresh model with specified initial parameters
    model = build_minimal_soen_model(init_j_in=init_j_in, init_j_out=init_j_out)
    model.train()
    
    # Loss and optimizer
    criterion = nn.MSELoss()
    optimizer = optimizer_class(model.parameters(), **optimizer_kwargs)
    
    # Tracking
    losses = []
    trajectory = [get_trainable_params(model)]  # Starting point
    
    for epoch in range(n_epochs):
        optimizer.zero_grad()
        
        # Forward pass
        final_hist, _ = model(X_data)
        
        # Take final time step as output
        output = final_hist[:, -1, :]  # [batch, 1]
        
        # Compute loss
        loss = criterion(output, y_data)
        
        # Backward pass
        loss.backward()
        
        # Gradient clipping
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        
        optimizer.step()
        
        losses.append(loss.item())
        trajectory.append(get_trainable_params(model))
        
        if verbose and (epoch + 1) % 20 == 0:
            j_in, j_out = trajectory[-1]
            print(f"  Epoch {epoch+1}: Loss={loss.item():.6f}, J_in={j_in:.4f}, J_out={j_out:.4f}")
    
    final_j_in, final_j_out = get_trainable_params(model)
    
    return {
        'losses': losses,
        'trajectory': trajectory,
        'final_j_in': final_j_in,
        'final_j_out': final_j_out,
        'model': model,
    }

## 5. Compare Optimizers

Train the 2-parameter SOEN model with different optimizers.

In [ ]:
# Optimizer configurations
OPTIMIZERS = {
    'SGD (lr=0.1)': (torch.optim.SGD, {'lr': 0.1}),
    'SGD (lr=0.5)': (torch.optim.SGD, {'lr': 0.5}),
    'SGD + Momentum': (torch.optim.SGD, {'lr': 0.1, 'momentum': 0.9}),
    'Adam (lr=0.05)': (torch.optim.Adam, {'lr': 0.05}),
    'Adam (lr=0.01)': (torch.optim.Adam, {'lr': 0.01}),
    'AdamW': (torch.optim.AdamW, {'lr': 0.05}),
    'RMSprop': (torch.optim.RMSprop, {'lr': 0.05}),
}

# Common starting point (away from optimum)
INIT_J_IN = 0.1
INIT_J_OUT = 0.8
N_EPOCHS = 100

# Train with each optimizer
results = {}
for name, (opt_class, opt_kwargs) in OPTIMIZERS.items():
    print(f"Training with {name}...")
    results[name] = train_soen_model(
        opt_class, opt_kwargs, X_data, y_data,
        n_epochs=N_EPOCHS, init_j_in=INIT_J_IN, init_j_out=INIT_J_OUT, verbose=False
    )
    print(f"  Final: J_in={results[name]['final_j_in']:.4f}, "
          f"J_out={results[name]['final_j_out']:.4f}, "
          f"Loss={results[name]['losses'][-1]:.6f}")

print(f"\nApproximate optimum: J_in={OPT_J_IN:.3f}, J_out={OPT_J_OUT:.3f}")

## 6. Visualize Loss Curves

In [ ]:
# Plot loss curves
fig, axes = plt.subplots(1, 2, figsize=(14, 5))
colors = plt.cm.tab10(np.linspace(0, 1, len(results)))

# Linear scale
ax1 = axes[0]
for (name, res), color in zip(results.items(), colors):
    ax1.plot(res['losses'], label=name, linewidth=2, color=color)
ax1.set_xlabel('Epoch')
ax1.set_ylabel('MSE Loss')
ax1.set_title('SOEN Training Loss (Linear Scale)')
ax1.legend(loc='upper right', fontsize=9)
ax1.grid(True, alpha=0.3)

# Log scale
ax2 = axes[1]
for (name, res), color in zip(results.items(), colors):
    ax2.plot(res['losses'], label=name, linewidth=2, color=color)
ax2.set_xlabel('Epoch')
ax2.set_ylabel('MSE Loss (log scale)')
ax2.set_yscale('log')
ax2.set_title('SOEN Training Loss (Log Scale)')
ax2.legend(loc='upper right', fontsize=9)
ax2.grid(True, alpha=0.3, which='both')

plt.tight_layout()
plt.show()

## 7. Visualize Optimization Trajectories

Show how each optimizer navigates the SOEN loss landscape in (J_in, J_out) parameter space.

In [ ]:
# Plot trajectories on contour
fig, ax = plt.subplots(figsize=(12, 10))

# Background contour
contour = ax.contour(J_IN, J_OUT, loss_surface, levels=30, cmap='gray', alpha=0.5)
ax.contourf(J_IN, J_OUT, loss_surface, levels=30, cmap='viridis', alpha=0.3)

# Plot each trajectory
for (name, res), color in zip(results.items(), colors):
    traj = np.array(res['trajectory'])
    ax.plot(traj[:, 0], traj[:, 1], '-', color=color, linewidth=2, label=name, alpha=0.8)
    ax.scatter(traj[0, 0], traj[0, 1], color=color, s=100, marker='o', edgecolor='black', zorder=5)
    ax.scatter(traj[-1, 0], traj[-1, 1], color=color, s=100, marker='s', edgecolor='black', zorder=5)

# Mark optimum and start
ax.scatter([OPT_J_IN], [OPT_J_OUT], color='red', s=200, marker='*', 
           zorder=10, label=f'Optimum ({OPT_J_IN:.2f}, {OPT_J_OUT:.2f})')
ax.scatter([INIT_J_IN], [INIT_J_OUT], color='black', s=150, marker='X', 
           zorder=10, label=f'Start ({INIT_J_IN}, {INIT_J_OUT})')

ax.set_xlabel('J_input (input weight)', fontsize=12)
ax.set_ylabel('J_output (output weight)', fontsize=12)
ax.set_title('SOEN Optimization Trajectories in Parameter Space', fontsize=14)
ax.legend(loc='upper left', fontsize=9)
ax.grid(True, alpha=0.3)
ax.set_xlim(J_IN_RANGE)
ax.set_ylim(J_OUT_RANGE)

plt.tight_layout()
plt.show()

## 8. Zoomed Trajectory Comparison

In [ ]:
# Individual trajectory plots
n_opts = len(results)
cols = 4
rows = (n_opts + cols - 1) // cols
fig, axes = plt.subplots(rows, cols, figsize=(16, 4*rows))
axes = axes.flatten() if n_opts > 1 else [axes]

for idx, ((name, res), color) in enumerate(zip(results.items(), colors)):
    ax = axes[idx]
    
    # Background
    ax.contourf(J_IN, J_OUT, loss_surface, levels=20, cmap='viridis', alpha=0.4)
    ax.contour(J_IN, J_OUT, loss_surface, levels=20, cmap='gray', alpha=0.3)
    
    # Trajectory
    traj = np.array(res['trajectory'])
    ax.plot(traj[:, 0], traj[:, 1], 'o-', color=color, linewidth=2, markersize=3, alpha=0.8)
    ax.scatter(traj[0, 0], traj[0, 1], color='black', s=80, marker='o', zorder=5)
    ax.scatter(traj[-1, 0], traj[-1, 1], color=color, s=80, marker='s', edgecolor='black', zorder=5)
    ax.scatter([OPT_J_IN], [OPT_J_OUT], color='red', s=100, marker='*', zorder=10)
    
    ax.set_title(f'{name}\nFinal loss: {res["losses"][-1]:.6f}', fontsize=10)
    ax.set_xlabel('J_input')
    ax.set_ylabel('J_output')
    ax.grid(True, alpha=0.3)

# Hide unused subplots
for idx in range(n_opts, len(axes)):
    axes[idx].set_visible(False)

plt.suptitle('Individual Optimizer Trajectories', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

## 9. Parameter Evolution Over Time

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# J_input evolution
ax1 = axes[0]
for (name, res), color in zip(results.items(), colors):
    traj = np.array(res['trajectory'])
    ax1.plot(traj[:, 0], label=name, color=color, linewidth=2)
ax1.axhline(y=OPT_J_IN, color='red', linestyle='--', linewidth=2, label=f'Optimum={OPT_J_IN:.2f}')
ax1.set_xlabel('Epoch')
ax1.set_ylabel('J_input')
ax1.set_title('Input Weight Evolution')
ax1.legend(loc='best', fontsize=8)
ax1.grid(True, alpha=0.3)

# J_output evolution
ax2 = axes[1]
for (name, res), color in zip(results.items(), colors):
    traj = np.array(res['trajectory'])
    ax2.plot(traj[:, 1], label=name, color=color, linewidth=2)
ax2.axhline(y=OPT_J_OUT, color='red', linestyle='--', linewidth=2, label=f'Optimum={OPT_J_OUT:.2f}')
ax2.set_xlabel('Epoch')
ax2.set_ylabel('J_output')
ax2.set_title('Output Weight Evolution')
ax2.legend(loc='best', fontsize=8)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 10. Summary Statistics

In [ ]:
import pandas as pd

# Create summary table
summary_data = []
for name, res in results.items():
    j_in_error = abs(res['final_j_in'] - OPT_J_IN)
    j_out_error = abs(res['final_j_out'] - OPT_J_OUT)
    param_dist = np.sqrt(j_in_error**2 + j_out_error**2)
    
    summary_data.append({
        'Optimizer': name,
        'Final Loss': f"{res['losses'][-1]:.6f}",
        'Final J_in': f"{res['final_j_in']:.4f}",
        'Final J_out': f"{res['final_j_out']:.4f}",
        'J_in Error': f"{j_in_error:.4f}",
        'J_out Error': f"{j_out_error:.4f}",
        'Dist to Opt': f"{param_dist:.4f}",
    })

df = pd.DataFrame(summary_data)
print("=" * 100)
print("SOEN OPTIMIZER COMPARISON SUMMARY")
print("=" * 100)
print(f"\nModel: 1D → 1D (SingleDendrite) → 1D")
print(f"Trainable parameters: 2 (J_input, J_output)")
print(f"Task: Linear regression through SOEN dynamics")
print(f"Approximate optimum: J_in={OPT_J_IN:.3f}, J_out={OPT_J_OUT:.3f}")
print(f"Starting point: J_in={INIT_J_IN}, J_out={INIT_J_OUT}")
print(f"Epochs: {N_EPOCHS}\n")
print(df.to_string(index=False))
print("=" * 100)

## 11. Convergence Speed Analysis

In [ ]:
def epochs_to_threshold(losses, threshold):
    """Return number of epochs to reach loss threshold."""
    for i, loss in enumerate(losses):
        if loss <= threshold:
            return i + 1
    return None

# Find reasonable thresholds based on results
min_final_loss = min(res['losses'][-1] for res in results.values())
thresholds = [0.1, 0.05, 0.02, 0.01, min_final_loss * 2]

print("Epochs to reach loss threshold:")
print("-" * 90)
header = f"{'Optimizer':<20}" + "".join([f"Loss<{t:.4f}  " for t in thresholds])
print(header)
print("-" * 90)

for name, res in results.items():
    row = f"{name:<20}"
    for thresh in thresholds:
        epochs = epochs_to_threshold(res['losses'], thresh)
        row += f"{str(epochs) if epochs else 'N/A':<14}"
    print(row)

## 12. Visualize Best Model's Predictions

In [ ]:
# Find best performing optimizer
best_name = min(results, key=lambda x: results[x]['losses'][-1])
best_model = results[best_name]['model']
print(f"Best optimizer: {best_name}")
print(f"Final loss: {results[best_name]['losses'][-1]:.6f}")

# Get predictions
best_model.eval()
with torch.no_grad():
    final_hist, all_hist = best_model(X_data)
    predictions = final_hist[:, -1, :].squeeze().numpy()

# Plot predictions vs targets
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Scatter plot
ax1 = axes[0]
ax1.scatter(y_data.squeeze().numpy(), predictions, alpha=0.6)
ax1.plot([y_data.min(), y_data.max()], [y_data.min(), y_data.max()], 'r--', linewidth=2)
ax1.set_xlabel('True Target')
ax1.set_ylabel('SOEN Prediction')
ax1.set_title(f'Predictions vs Targets ({best_name})')
ax1.grid(True, alpha=0.3)

# Input vs output relationship
ax2 = axes[1]
x_means = X_data.mean(dim=1).squeeze().numpy()
ax2.scatter(x_means, y_data.squeeze().numpy(), alpha=0.5, label='True targets')
ax2.scatter(x_means, predictions, alpha=0.5, label='SOEN predictions')
ax2.set_xlabel('Input flux (mean)')
ax2.set_ylabel('Output')
ax2.set_title('SOEN Model: Input-Output Relationship')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 13. Key Observations

### Model Summary
- **Architecture**: 1D → 1D (SingleDendrite) → 1D
- **Trainable parameters**: Exactly 2 (J_input, J_output)
- **Task**: Linear regression through SOEN neuron dynamics

### SingleDendrite Dynamics
The neuron evolves according to:
$$\frac{ds}{dt} = \gamma^+ \cdot g(\phi_{eff}) - \gamma^- \cdot s$$

where:
- $\phi_{eff} = J_{input} \cdot x + \phi_{offset}$ (effective input flux)
- $g(\cdot)$ is the source function (Heaviside fit)
- Output = $J_{output} \cdot s_{final}$

### Optimizer Behavior on SOEN
| Optimizer | Characteristics |
|-----------|----------------|
| **SGD** | Predictable path, may be slow |
| **SGD + Momentum** | Faster convergence, smoother trajectory |
| **Adam** | Adaptive learning, good for SOEN's varying gradients |
| **AdamW** | Adam with weight decay regularization |
| **RMSprop** | Good for non-stationary objectives |

### Physical Mapping
This 1-neuron model would map to **1 physical SingleDendrite neuron** on SOEN hardware.

In [ ]:
print("Notebook complete!")
print(f"\nBest optimizer: {best_name}")
print(f"Final parameters: J_in={results[best_name]['final_j_in']:.4f}, J_out={results[best_name]['final_j_out']:.4f}")
print(f"Final loss: {results[best_name]['losses'][-1]:.6f}")